### Assignment 9

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [16]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

In [6]:
inst = pd.read_csv(r'../../_data/institutos1.csv')

inst_geo = gpd.GeoDataFrame (inst, crs = "EPSG:4326",
                            geometry = gpd.points_from_xy( inst.nlong_ie,
                                                         inst.nlat_ie))
shp_dpt = gpd.read_file(r"..\..\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp")

intersect_inst_geo = gpd.overlay(inst_geo,shp_dpt, how = 'intersection')

vars_sum = ['ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm',
           'sumaden_flagingresante','sumaden_flagmatriculado']
tot_dpt = intersect_inst_geo.groupby(['CCDD'], as_index = False)[vars_sum].sum()

tot_dpt_shp = shp_dpt.merge(tot_dpt, on = 'CCDD')

In [7]:
inst

,cod_mod,ltimoden_metaatencion,cuentadeid_postulante_procesoadm,sumaden_flagingresante,sumaden_flagmatriculado,ratio,ratio1,dif,dif1,nlat_ie,...,altitude,Dblock1km,Dblock2km,Dblock3km,Dblock5km,Dblock10km,Dblock20km,Dblock30km,_merge,geometry
0,207597,307,481,302,301,0.980456,1.566775,-6,174,-13.52107,...,3386.0,14840369,742184,494123,296073,148036,7418,4912,matched (3),POINT (-71.97748 -13.52107)
1,207613,240,502,204,204,0.850000,2.091667,-36,262,-13.63997,...,2342.0,14970470,748235,499156,299094,149047,7423,4915,matched (3),POINT (-72.88261 -13.63997)
2,239970,537,1047,484,476,0.886406,1.949721,-61,510,-15.47827,...,3831.0,17000163,850081,566054,340032,170016,8508,5605,matched (3),POINT (-70.12756 -15.47827)
3,262311,423,526,353,349,0.825059,1.243499,-74,103,-6.23041,...,2358.0,6781025,339512,226341,135205,67102,3351,2234,matched (3),POINT (-77.86943 -6.23041)
4,273979,425,544,411,405,0.952941,1.280000,-20,119,-6.49004,...,309.0,7070857,353428,235285,141171,70085,3542,2328,matched (3),POINT (-76.36539 -6.49004)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1468594,1566,250,241,202,0.128991,0.159642,-1364,-1316,-12.02987,...,125.0,13190935,659467,439311,263187,131093,6546,4331,matched (3),POINT (-77.05831 -12.02987)
211,1468610,284,74,71,71,0.250000,0.260563,-213,-210,-12.08215,...,115.0,13250934,662467,441311,265186,132093,6646,4431,matched (3),POINT (-77.05295 -12.08215)
212,1545623,210,266,184,177,0.842857,1.266667,-33,56,-16.13940,...,3780.0,17740208,887104,591069,354041,177020,8810,5906,matched (3),POINT (-70.53580 -16.13940)
213,1595495,206,144,141,137,0.665049,0.699029,-69,-62,-18.01370,...,577.0,19810177,990088,660059,396035,198017,9908,6605,matched (3),POINT (-70.25080 -18.01370)


In [14]:
inst.columns

Index(['cod_mod', 'ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm',
       'sumaden_flagingresante', 'sumaden_flagmatriculado', 'ratio', 'ratio1',
       'dif', 'dif1', 'nlat_ie', 'nlong_ie', 'population_ccpp', 'altitude',
       'Dblock1km', 'Dblock2km', 'Dblock3km', 'Dblock5km', 'Dblock10km',
       'Dblock20km', 'Dblock30km', '_merge', 'geometry'],
      dtype='object')

## Make an interactive map of markes using visual_html function and show total vacancies, applicants, entrants, and enrolled information in a table popup. Additionally, use "university" as icon.

In [24]:
def visual_html(i):
   
    Total_Vacancies = inst['ltimoden_metaatencion'].iloc[i]                            
    Total_Applicants = inst['cuentadeid_postulante_procesoadm'].iloc[i]
    Total_Entrants = inst['sumaden_flagingresante'].iloc[i]  
    Total_Enrolled = inst['sumaden_flagmatriculado'].iloc[i]  
    
    
    left_col_colour = "#FA8072"
    right_col_colour = "#BDC3C7"
    
    html = """<!DOCTYPE html>
<html>

<head>
    <p> Institutes </p>

</head>
    <table style="height: 126px; width: 350px;">  <!-- Comment: Create a teable. -->

<!-- Add information  -->

<tbody> 
<tr>

<!-- Add color by column -->

<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Vacancies</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Vacancies) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Applicants</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Applicants) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Entrants</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Entrants) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Enrolled</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Enrolled) + """
</tr>


</tbody>
</table>
</html>
"""
    return html

In [25]:
ubication = inst['nlat_ie'].mean(), inst['nlong_ie'].mean()  # Average point

inst2 = fm.Map(location = ubication, zoom_start=12)

for i in range(0,len(inst)):
    html = visual_html(i)

    iframe = br.element.IFrame(html=html,width=350,height=300)
    popup = fm.Popup(iframe, parse_html=True)
    
    fm.Marker([inst['nlat_ie'].iloc[i],inst['nlong_ie'].iloc[i]],
                  popup=popup, icon=fm.Icon(color= 'blue', icon='university', prefix="fa")).add_to(inst2)

inst2.save("institutes.html")
inst2